### Startup

In [1]:
# Load openai api key from .env file

import os
from dotenv import load_dotenv
load_dotenv("../../../../.env")

# First way to patch openai client directly

# from openai import OpenAI
# from gai.llm.openai import patch_chatcompletions
# client = patch_chatcompletions(OpenAI())

# Preferred way is to use the patched wrapper

from gai.llm.openai import OpenAI
client = OpenAI()


---

### Generate

In [3]:
from openai import OpenAI

model = "gpt-4o-mini"

## Using openai client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = OpenAI()
response = client.chat.completions.create(
    model=model, 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)

print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.choices[0].message.content}")


## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model=model, 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)

print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.extract()}")

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI()
response = await client.chat.completions.create(
    model=model, 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)

print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.extract()}")



Normal:

Raw:
{'id': 'chatcmpl-Bgn1mgRdHBHseYGl239cCpzdJXLzF', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'As the last leaf fell from the ancient oak, a whisper of autumn filled the air with memories of laughter, love, and a promise of new beginnings.', 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': None}}], 'created': 1749537938, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_34a54ae93c', 'usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}
Extracted:
As the last leaf fell from the ancient oak, a whisper of autumn filled the air with memories of laughter, love, and a promise of new

---
### Stream

In [4]:
from openai import OpenAI

model = "gpt-4o-mini"

## Using openai client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

import json
client = OpenAI()
response = client.chat.completions.create(
    model=model, 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
for chunk in response:
    #chunks.append(chunk.model_dump())
    chunks.append(chunk.choices[0].delta.content)
print(json.dumps(chunks,indent=4))

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model=model, 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
for chunk in response:
    chunk = chunk.extract()
    if (isinstance(chunk, str)):
        print(chunk, end="", flush=True)

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI()
response = await client.chat.completions.create(
    model=model, 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
async for chunk in response:
    chunk = chunk.extract()
    if (isinstance(chunk, str)):
        print(chunk, end="", flush=True)




Normal:

[
    "",
    "As",
    " the",
    " last",
    " leaf",
    " fell",
    " from",
    " the",
    " ancient",
    " oak",
    ",",
    " a",
    " whispered",
    " promise",
    " echoed",
    " in",
    " the",
    " crisp",
    " autumn",
    " air",
    ",",
    " reminding",
    " her",
    " that",
    " even",
    " endings",
    " could",
    " be",
    " beautiful",
    " beginnings",
    ".",
    null
]


Patched:

Under the flickering streetlamp, she found a note etched with a promise, revealing that every lost moment had led her to this perfect night.

Async:

Under the glow of a forgotten star, a lonely traveler discovered an ancient map leading to a hidden world where dreams and reality intertwined like the roots of an old tree.

---
### Tool Call

In [5]:
from openai import OpenAI

model = "gpt-4o-mini"

## Using openai client ────────────────────────────────────────────────────────────────────────

print("\n\nUnpatched:\n")

openai_client = OpenAI()
response = openai_client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required"
)
import json
print(json.dumps(response.dict(), indent=4))

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required"
)
print(response.extract().model_dump_json(indent=4))

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI()
response = await client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required"
)
print(response.extract().model_dump_json(indent=4))



Unpatched:



/tmp/ipykernel_40418/638719575.py:35: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(response.dict(), indent=4))


{
    "id": "chatcmpl-Bgn3NI0paJYjQGKt8TEdNAik4jWJg",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "annotations": [],
                "audio": null,
                "function_call": null,
                "tool_calls": [
                    {
                        "id": "call_Erh7f4NhiOEkCeAEZ9h9MQZx",
                        "function": {
                            "arguments": "{\"search_query\":\"current time in Singapore\"}",
                            "name": "google"
                        },
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1749538037,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint":

Parameterless tool call

In [23]:
from openai import OpenAI
import json

model = "gpt-4o-mini"

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get current time in Singapore",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ],
    tool_choice="required"
)
import json
print(response.extract().model_dump_json(indent=4))

{
    "tool_name": "get_current_time",
    "arguments": "{}"
}


Can't think of a good reason why tool call should be streamed but here it is provided to maintain compatibility with the original API.



In [24]:
from openai import OpenAI
import json

model = "gpt-4o-mini"

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    stream=True,
)

for chunk in response:
    chunk = chunk.extract()
    print("extracted:",chunk)




Patched:

extracted: 
extracted: 
extracted: 
extracted: 
extracted: 
extracted: 
extracted: 
extracted: 
extracted: 
extracted: 
extracted: type='finish_reason' finish_reason='tool_calls' tool_name='google' arguments='{"search_query":"current time in Singapore"}'


When tool call is not needed.

In [25]:
from openai import OpenAI
import json

model = "gpt-4o-mini"

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Tell me a one sentence story"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    stream=True,
)

for chunk in response:
    chunk = chunk.extract()
    print("extracted",chunk)




Patched:

extracted 
extracted As
extracted  the
extracted  last
extracted  leaf
extracted  fell
extracted  from
extracted  the
extracted  ancient
extracted  oak
extracted ,
extracted  the
extracted  little
extracted  girl
extracted  whispered
extracted  her
extracted  final
extracted  goodbye
extracted ,
extracted  promising
extracted  to
extracted  return
extracted  and
extracted  share
extracted  her
extracted  secrets
extracted  with
extracted  the
extracted  whisper
extracted ing
extracted  winds
extracted .
extracted type='finish_reason' finish_reason='stop'


---

## Structured Output

In [6]:
import json
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int
data = """Foundation is a science fiction novel by American writer
        Isaac Asimov. It is the first published in his Foundation Trilogy (later
        expanded into the Foundation series). Foundation is a cycle of five
        interrelated short stories, first published as a single book by Gnome Press
        in 1951. Collectively they tell the early story of the Foundation,
        an institute founded by psychohistorian Hari Seldon to preserve the best
        of galactic civilization after the collapse of the Galactic Empire."""

## Using openai client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

from openai import OpenAI
client = OpenAI()
response = client.beta.chat.completions.parse(
    model="gpt-4o", 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
print(response.model_dump_json(indent=4))

## Using gai.llm.openai client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI()
response = client.beta.chat.completions.parse(
    model="gpt-4o", 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
dict = response.model_dump(exclude={"extract"})
print(json.dumps(dict, indent=4))

print("\n\nExtracted:\n")
print(response.extract())

## Using gai.llm.openai client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI()
response = await client.beta.chat.completions.parse(
    model="gpt-4o", 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
dict = response.model_dump(exclude={"extract"})
print(json.dumps(dict, indent=4))

print("\n\nExtracted:\n")
print(response.extract())





Normal:

{
    "id": "chatcmpl-Bgn4fcK6Vt3tVdNlxnOnJ0Gq21Wqj",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "{\"title\":\"Foundation\",\"summary\":\"\\\"Foundation\\\" is a seminal science fiction novel by Isaac Asimov that explores the rise and fall of a grand galactic empire through a blend of psychohistorical insights. Set in a distant future where humanity has spread across the Milky Way, the story revolves around Hari Seldon, a visionary psychohistorian. As the declining Galactic Empire faces inevitable collapse, Seldon predicts a 30,000-year dark age based on the science of psychohistory, a theoretical blend of history, sociology, and statistical mathematics. \\n\\nTo mitigate this grim future, Seldon establishes the \\\"Foundation,\\\" a repository of humanity's collective knowledge aimed at accelerating the rebirth of a new and vibrant civilization. The